### Try to build a an agent that will work with client to compile and execute a sql order
 - bonus: build a tool to email the order details (including price) when done

In [1]:
# open ai authentication

import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
import dbio

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [3]:
# email util

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from langchain.agents import tool

def get_mail_server(server='smtp.gmail.com', port=587):
    '''Allocate mail server on port'''
    mailServer = smtplib.SMTP(server, port)
    test = mailServer.starttls()  # start TLS
    assert test[1].lower().find(b'ready') != -1
    return mailServer

def format_message(message, sender, receiver, subject):
    '''
    Format arguments into MIMEMultipart object
    '''

    # message
    msg = MIMEMultipart()
    msg['From'] = sender
    msg['To'] = ', '.join(receiver)
    msg['Subject'] = subject
    msg.attach(MIMEText(message, 'html'))

    return msg

def send_message(order_summary, email='user@user.com'):
    """
    Send the order summary (from the order summary tool)
    to the email indicated by the user
    """
    
    sender = 'OrderBotPizza.com'
    subject = 'Your OrderBot Pizza Order'
    verbose = True

    msg = format_message(order_summary, sender, [email], subject)
    '''
    if verbose:
        print(f'sending mail from {sender} to {receiver}')
    with get_mail_server() as mailServer:
        mailServer.sendmail(sender, receiver, msg.as_string())
    '''
    print(msg)
    return msg, 'mail sent successfully'

In [ ]:
send_message('hello time')

#enabling gmail will require 2-factor authentication with google

# Custom Agent w/ pizza menu in prompt

 - https://python.langchain.com/docs/modules/agents/how_to/custom_llm_agent
 - See if order can be taken, summarized accurately, and then emailed
 - Test for hallucinations, if wrong output try incorporating db

In [ ]:
# setup env

from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, LLMMathChain, LLMChain
from langchain.chat_models import ChatOpenAI
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, OutputParserException
import re

In [ ]:
# llm
llm = ChatOpenAI(temperature=0)

# set up tool
# Define which tools the agent can use to answer user queries
tools = [
    # Tool(
    #    name="Calculator",
    #    func=LLMMathChain.from_llm(llm=llm, verbose=True).run,
    #    description="useful for when you need to perform math operations"
    #),
        Tool(
            name = "send email",
            func=send_message,
            description="""Used when sending the use the summary email of the order. Call this only when the user \
            has finished entering their order and provided an email address."""
    ),
]

In [ ]:
# prompt template

# Set up the base template
template_with_history = """
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment and send an email record of the order.\

Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\

You respond in a short, very conversational friendly style. \

The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \

You have access to the following tools for sending the final summary email:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, may be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer (OR I CANNOT ANSWER THE QUESTION)
Final Answer: the final answer to the original input question

Begin! Remember to call the email summary tool only when you know the \
recipient's email address.

Previous conversation history:
{history}

New question: {input}
{agent_scratchpad}"""

# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

prompt_with_history = CustomPromptTemplate(
    template=template_with_history,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps", "history"]
)

In [ ]:
# output parser

class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise OutputParserException(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

output_parser = CustomOutputParser()

Define the stop sequence

This is important because it tells the LLM when to stop generation.

This depends heavily on the prompt and model you are using. Generally, you want this to be whatever token you use in the prompt to denote the start of an Observation (otherwise, the LLM may hallucinate an observation for you).

In [ ]:
# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt_with_history)

tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names,
)

In [ ]:
# memory - how big is this buffer?

from langchain.memory import ConversationBufferWindowMemory
memory=ConversationBufferWindowMemory(k=25)

In [ ]:
# use agent

agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, memory=memory, 
                                                    verbose=True, max_iterations=3)

agent_executor.run("Hello my name is Tim")

# try with conversational agent and sql tool

 - reference for agent that applies custom output parser: https://github.com/hwchase17/langchain/issues/2561

 - following the example [here](https://python.langchain.com/en/latest/modules/agents/agents/examples/chat_conversation_agent.html) of an agent that can call tools and chat with the usert too
 - Note there is no prompt specific setting the context for the OrderBot

 - Try adding llm-math as tool

 - QA chain from text document for plain text version of prompt

 - general custom llm agent is here: https://python.langchain.com/docs/modules/agents/how_to/custom_llm_agent
 - 

In [4]:
from langchain.agents import Tool, AgentOutputParser
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.sql_database import SQLDatabase
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain import LLMMathChain

from langchain.schema import AgentAction, AgentFinish, OutputParserException
from langchain.output_parsers.json import parse_json_markdown

from typing import Union

In [5]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [6]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, verbose=True)

In [7]:
db = SQLDatabase.from_uri("sqlite:///./pizza_v2.db")
from langchain.chains import SQLDatabaseChain, SQLDatabaseSequentialChain
seq_chain = SQLDatabaseSequentialChain.from_llm(llm, db, verbose=True)

# possibly customize SQLDatabaseChain to handle empty table list from SQLDatabaseSequentialChain
# then pass SQLDatabaseChain into the sequential chain

#seq_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [8]:
tools = [
    Tool(
        name = "menu search",
        func=seq_chain.run,
        description="""useful for when you need to answer questions about menu items the user wants to order.
        Search your chat history before deciding you need to use this."""
    ),

        Tool(
        name="Order summary",
        func=lambda x: "Summarize the order including total cost",
        description="useful for producing a summary of the order",
    ),

    # chain that first summarizes and them emails
    # maybe need a chain of calls that get email, order summary, then write the email

        Tool(
        name = "send email",
        func=send_message,
        description="""Used when sending the use the email of the order. The input is the output of the order summary tool."""
    ),
]

In [9]:
# Custom output parser
class CustomConvoOutputParser(AgentOutputParser):
    def get_format_instructions(self) -> str:
        return FORMAT_INSTRUCTIONS

    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        try:
            response = parse_json_markdown(text)
            action, action_input = response["action"], response["action_input"]
            if action == "Final Answer":
                return AgentFinish({"output": action_input}, text)
            else:
                return AgentAction(action, action_input, text)
        except Exception as e:
            # Return instead of throwing an error
            return AgentFinish({"output": action_input}, text)

    @property
    def _type(self) -> str:
        return "conversational_chat"

In [10]:
agent_chain = initialize_agent(tools, llm, agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
                               verbose=True, memory=memory)
agent_chain.agent.output_parser = CustomConvoOutputParser()

# try override here
#agent_chain.agent.llm_chain.prompt = ChatPromptTemplate goes here, put details in system role if possible

# presumably need to define steps so that 1) order taken, 2) summary created, 3) get user email if not already known, 
# 4) email summary is then sent

# access chat memory with agent_chain.memory

In [ ]:
agent_chain.memory.chat_memory.json()

In [ ]:
#agent_chain.agent.output_parser??

In [11]:
agent_chain.run(input="hi, i am Tim and my email is t@gmail.com")



> Entering new  chain...
{
    "action": "Final Answer",
    "action_input": "Hello Tim! How can I assist you today?"
}

> Finished chain.


'Hello Tim! How can I assist you today?'

In [ ]:
agent_chain.memory

In [ ]:
agent_chain.run(input="what is my name and email")

In [ ]:
agent_chain.run('I would like to order a large pizza and bottled water')

In [ ]:
agent_chain.run('What sizes of bottled water do you have?')

In [ ]:
#agent_chain.run('I will take the bottled water.  What is you most popular pizza?')
agent_chain.run('Add the large bottled water to my order.')

In [ ]:
agent_chain.run('Add the bottled water to my order.')

In [12]:
agent_chain.run(input="""I would like to order the large pepperoni \
pizza. Please include both sausage and extra cheese as additional \
toppings.
""")



> Entering new  chain...
{
    "action": "menu search",
    "action_input": "large pepperoni pizza with sausage and extra cheese"
}

> Entering new  chain...


/home/timw/venv/llm/lib/python3.11/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Table names to use:
['pizzas', 'toppings']

> Entering new  chain...
large pepperoni pizza with sausage and extra cheese
SQLQuery:SELECT pizzas.Item, pizzas.Price_Large, toppings.Price
FROM pizzas, toppings
WHERE pizzas.Item = 'Pepperoni pizza' AND toppings.Item IN ('Sausage', 'Extra cheese')
SQLResult: [('Pepperoni pizza', '12.95', '2.00'), ('Pepperoni pizza', '12.95', '3.00')]
Answer:The price of a large pepperoni pizza with sausage and extra cheese is $17.95 (12.95 + 2.00 + 3.00).
> Finished chain.

> Finished chain.

Observation: The price of a large pepperoni pizza with sausage and extra cheese is $17.95 (12.95 + 2.00 + 3.00).
Thought:{
    "action": "Final Answer",
    "action_input": "The price of a large pepperoni pizza with sausage and extra cheese is $17.95 (12.95 + 2.00 + 3.00)."
}

> Finished chain.


'The price of a large pepperoni pizza with sausage and extra cheese is $17.95 (12.95 + 2.00 + 3.00).'

In [13]:
agent_chain.run(input="Please repeat my order to make sure it is right.")



> Entering new  chain...
{
    "action": "Order summary",
    "action_input": "t@gmail.com"
}
Observation: Summarize the order including total cost
Thought:{
    "action": "Final Answer",
    "action_input": "Your order is a large pepperoni pizza with sausage and extra cheese, and the total cost is $17.95."
}

> Finished chain.


'Your order is a large pepperoni pizza with sausage and extra cheese, and the total cost is $17.95.'

In [ ]:
agent_chain.run(input="how many kinds of fish do you sell")

In [14]:
agent_chain.run(input="that is it. please email me my receipt")



> Entering new  chain...
```json
{
    "action": "send email",
    "action_input": "Here is your order summary:\n\nLarge Pepperoni Pizza with Sausage and Extra Cheese\nTotal Cost: $17.95\n\nThank you for your order!"
}
```Content-Type: multipart/mixed; boundary="===============7057915733348356565=="
MIME-Version: 1.0
From: OrderBotPizza.com
To: user@user.com
Subject: Your OrderBot Pizza Order

--===============7057915733348356565==
Content-Type: text/html; charset="us-ascii"
MIME-Version: 1.0
Content-Transfer-Encoding: 7bit

Here is your order summary:

Large Pepperoni Pizza with Sausage and Extra Cheese
Total Cost: $17.95

Thank you for your order!
--===============7057915733348356565==--


Observation: (<email.mime.multipart.MIMEMultipart object at 0x7f0c796a2e10>, 'mail sent successfully')
Thought:```json
{
    "action": "Final Answer",
    "action_input": "Your order summary has been sent to your email. Thank you for your order!"
}
```

> Finished chain.


'Your order summary has been sent to your email. Thank you for your order!'